In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.misc import imread
import matplotlib.image as mpimg
import os

In [2]:
def load_all_labels(data_path):
    raw_labels = []
    with open(data_path) as input_labels:
        for line in input_labels:
            tokens = line.strip().split(',')
            raw_labels.append(tokens[1])
    return raw_labels

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
raw_labels = load_all_labels('data/train_v2.csv')
raw_labels = raw_labels[1:]
label_tf = CountVectorizer(binary=True, max_features=20)
labels = label_tf.fit_transform(raw_labels)

In [4]:
labels

<40479x17 sparse matrix of type '<class 'numpy.int64'>'
	with 116205 stored elements in Compressed Sparse Row format>

In [5]:
label_tf.get_feature_names()

['agriculture',
 'artisinal_mine',
 'bare_ground',
 'blooming',
 'blow_down',
 'clear',
 'cloudy',
 'conventional_mine',
 'cultivation',
 'habitation',
 'haze',
 'partly_cloud',
 'primary',
 'road',
 'selective_logging',
 'slash_burn',
 'water']

In [6]:
n_samples = labels.shape[0]
indices = np.random.permutation(n_samples)
#Number of images for train dataset and test dataset
n_train = 1000
n_test = 2000
# split the train and test 
#n_test = int(n_samples * 0.3)
print("Numbers of total images: ", n_samples)
print("Numbers of training images: ", n_train)
print("Number of test images: ", n_test)
#test = train_img[indices[:n_test]]
#train = train_img[indices[n_test:]]
test_labels = labels[indices[:n_test]]
train_labels = labels[indices[n_test:n_train+n_test]]

Numbers of total images:  40479
Numbers of training images:  1000
Number of test images:  2000


In [7]:
def load_image(data_path, indices, image_size):
    num_images = 0
    dataset = np.ndarray(shape=(len(indices), image_size, image_size, 3),dtype=np.float32)
    for i in tqdm(range(len(indices)), total = len(indices)):
        image = data_path + 'train_' + str(indices[i]) + '.jpg'
        image_data = mpimg.imread(image)[:,:,:3]
        #Normalization
        image_data = np.multiply(image_data, 1.0/255.0)
        dataset[num_images, :, :, :] = image_data
        num_images += 1
    print('Full dataset tensor:', dataset.shape)
    return dataset

In [9]:
data_path = "data/train-jpg/"
image_size = 256
print("Loading training images data...")
train = load_image(data_path, indices[n_test:n_train+n_test], image_size)
print("Loading testing images data...")
test = load_image(data_path, indices[:n_test], image_size)

  1%|          | 6/1000 [00:00<00:16, 59.32it/s]

Loading training images data...


  1%|          | 16/2000 [00:00<00:12, 156.88it/s]

Full dataset tensor: (1000, 256, 256, 3)
Loading testing images data...


100%|██████████| 2000/2000 [00:12<00:00, 159.62it/s]

Full dataset tensor: (2000, 256, 256, 3)


In [10]:
print(train.shape)
print(train_labels.shape)
print(test.shape)
print(test_labels.shape)

(1000, 256, 256, 3)
(1000, 17)
(2000, 256, 256, 3)
(2000, 17)


In [11]:
# Compute KL divergence
test_disb = np.sum(test_labels, axis=0)
train_disb = np.sum(train_labels, axis=0)
test_disb = test_disb / float(np.sum(test_disb))
train_disb = train_disb / float(np.sum(train_disb))

kl = np.sum(np.multiply(train_disb, (np.log(train_disb) - np.log(test_disb))))
print(kl)

kl = np.sum(np.multiply(test_disb, (np.log(test_disb) - np.log(train_disb))))
print(kl)

0.00893963997073
0.00831867467262


In [12]:
import os
import tensorflow as tf

#os.environ["CUDA_VISIBLE_DEVICES"]="3"

def get_session(gpu_fraction=0.1):
    '''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''
    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
    
import keras.backend as K
#K.set_session(get_session(1.0))

Using TensorFlow backend.


In [18]:
import os
import tensorflow as tf
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

batch_size = 32
num_classes = 17
epochs = 20
data_augmentation = False

x_train = train
x_test = test
y_train = train_labels.toarray()
y_test = test_labels.toarray()

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (1000, 256, 256, 3)
1000 train samples
2000 test samples


In [19]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)

In [20]:
def accuracy_with_threshold(y_true, y_pred):
    threshold = 0.5
    y_pred = K.cast(K.greater(y_pred, threshold), K.floatx())
    return K.mean(K.equal(y_true, y_pred))

def hamming_dist(y_true, y_pred):
    threshold = 0.5
    y_pred = K.cast(K.greater(y_pred, threshold), K.floatx())
    return K.mean(K.sum(K.abs(y_true - y_pred), axis=1))

In [21]:
# Let's train the model using RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=[accuracy_with_threshold, hamming_dist])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

if not data_augmentation:
    print('Not using data augmentation.')
    hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                     validation_data=(x_test, y_test), shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))

Not using data augmentation.
Train on 1000 samples, validate on 2000 samples
Epoch 1/20
1000/1000 [==============================] - 862s - loss: 0.4838 - accuracy_with_threshold: 0.8754 - hamming_dist: 2.1190 - val_loss: 0.2655 - val_accuracy_with_threshold: 0.9063 - val_hamming_dist: 1.5935

In [22]:
print(hist.history)

{'loss': [0.48383314275741579, 0.27903114080429076, 0.26692418193817141, 0.25143528723716735, 0.25281196451187132, 0.24181845676898955, 0.23834360158443452, 0.23155634951591492, 0.2312601249217987, 0.22988275420665741, 0.22637898385524749, 0.22026603877544404, 0.22591736078262328, 0.21504541945457459, 0.21378689765930176, 0.20335748410224916, 0.20658554005622864, 0.20326781964302063, 0.19321963787078858, 0.18575949239730835], 'hamming_dist': [2.1190000000000002, 1.72, 1.657, 1.6200000000000001, 1.6379999999999999, 1.5660000000000001, 1.5860000000000001, 1.538, 1.5269999999999999, 1.5329999999999999, 1.52, 1.4530000000000001, 1.478, 1.403, 1.4279999999999999, 1.3700000000000001, 1.347, 1.3480000000000001, 1.278, 1.2010000000000001], 'val_loss': [0.2654597638845444, 0.25783159828186036, 0.2725900321006775, 0.31582064557075501, 0.23698623919487, 0.23177436435222626, 0.22561672949790954, 0.22477884781360627, 0.24523217070102693, 0.21382023429870606, 0.21903195524215699, 0.26395094609260561

In [23]:
y_pred = model.predict(x_test, batch_size=32)

In [36]:
y_pred.shape

(2000, 17)

In [41]:
y_pred = K.cast(K.greater(y_pred, 0.5), K.floatx())
sess = tf.InteractiveSession()
y_pred = y_pred.eval()
y_pre = K.mean(K.equal(y_test, y_pred))
y_pre = y_pre.eval()
print(y_pre)

0.926088


In [43]:
label_tf.get_feature_names()

['agriculture',
 'artisinal_mine',
 'bare_ground',
 'blooming',
 'blow_down',
 'clear',
 'cloudy',
 'conventional_mine',
 'cultivation',
 'habitation',
 'haze',
 'partly_cloudy',
 'primary',
 'road',
 'selective_logging',
 'slash_burn',
 'water']

In [44]:
indices[:10]

array([29473,  5907, 15586, 13899, 13543, 24055, 37775,  7774, 32957, 10629])

In [45]:
y_test[0]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [47]:
labels_a = labels.toarray()

In [49]:
labels_a[29473]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [51]:
y_pred[0]

array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.], dtype=float32)

In [57]:
with open('output/output-indices.txt', 'w') as output:
    output.write("shaffled index-indices:" + '\n')
    for line in indices:
        output.write(str(line)+'\n')
        
with open('output/output-y_train.txt', 'w') as output:
    output.write("train labels-y_train:" + '\n')
    for line in y_train:
        output.write(str(line)+'\n')
        
with open('output/output-y_test.txt', 'w') as output:
    output.write("train labels-y_test:" + '\n')
    for line in y_test:
        output.write(str(line)+'\n')

In [58]:
with open('output/output-y_pred.txt', 'w') as output:
    output.write("predict labels-y_pred:" + '\n')
    for line in y_pred:
        output.write(str(line)+'\n')

In [62]:
with open('output/output-indices_test.txt', 'w') as output:
    output.write("shaffled index-indices_test:" + '\n')
    for line in indices[:n_test]:
        output.write(str(line)+'\n')

In [63]:
with open('output/output-indices_train.txt', 'w') as output:
    output.write("shaffled index-indices_train:" + '\n')
    for line in indices[n_test:n_test+n_train]:
        output.write(str(line)+'\n')